In [1]:
import os

import dotenv
from langchain.agents import create_sql_agent
from langchain.utilities import SQLDatabase
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

# from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType

In [2]:
dotenv.load_dotenv()

True

In [3]:
db = SQLDatabase.from_uri(os.getenv("DB_CONNECT"))

llm = ChatOpenAI(temperature=0, model="gpt-4-1106-preview")
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [ ]:
agent_executor.run("What game did Luka Doncic have the highest three pointers percentage?")